# <center>Implementing LDA in Python</center>

## Key Concepts in this Notebook

## Introduction

## Importing the Required Libraries

In [2]:
#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#1introduction
#import nltk 
#nltk.download("stopwords")


import numpy as np
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#Spacy
#!pip install spacy
#!python -m spacy download en_core_web_lg 
import spacy
#import en_core_web_sm
from nltk.corpus import stopwords

#vis
#!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim
#or import pyLDAvis.gensim_models instead of pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category= DeprecationWarning)

## Preparing Data

In [3]:
def load_data(file):
    with open (file, "r", encoding= "utf-8") as f:
        data = json.load(f)
    return(data)

def write_data(file):
    with open (file, "w", encoding = "utf-8") as f:
        json.dump(data, f, indent=4)

In [4]:
stopwords = stopwords.words("english")

In [5]:
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [6]:
data = load_data("Data/ushmm_dn.json")["texts"]

print(data[0][0:90])

 My name David Kochalski. I was born in a small town called , and I was born May 5, 1928. 


In [7]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)

lemmatized_texts = lemmatization(data)
print(lemmatized_texts[0][0:90])

name bear small town call bear very hard work child father mother small mill flour buckwhe


In [8]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc= True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print(data_words[0][0:20])

['name', 'bear', 'small', 'town', 'call', 'bear', 'very', 'hard', 'work', 'child', 'father', 'mother', 'small', 'mill', 'flour', 'buckwheat', 'prosperous', 'comfortable', 'go', 'school']


In [41]:
#Bigrams and Trigrams
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold = 100)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold = 100)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)
print(data_bigrams_trigrams[0][0:20])

['name', 'bear', 'small', 'town', 'call', 'bear', 'very', 'hard', 'work', 'child', 'father', 'mother', 'small', 'mill', 'flour', 'buckwheat', 'prosperous', 'comfortable', 'go', 'school']


In [38]:
#TF-IDF Removal
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts] 
print(corpus[0][0:20])

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words = []
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = []
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow] 
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words + words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids]
    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow




[(0, 1), (1, 1), (2, 11), (3, 1), (4, 2), (5, 1), (6, 2), (7, 1), (8, 2), (9, 3), (10, 1), (11, 12), (12, 1), (13, 8), (14, 1), (15, 2), (16, 1), (17, 3), (18, 2), (19, 1)]


In [9]:
#id2word = corpora.Dictionary(data_words)
#
#corpus=[]
#
#for text in data_words:
#    new = id2word.doc2bow(text)
#    corpus.append(new)
#    
#print(corpus[0][0:20])
#
#word = id2word[[0][:1][0]]
#print(word)

[(0, 2), (1, 11), (2, 1), (3, 2), (4, 1), (5, 2), (6, 1), (7, 2), (8, 3), (9, 1), (10, 12), (11, 1), (12, 8), (13, 1), (14, 2), (15, 1), (16, 3), (17, 2), (18, 1), (19, 2)]
able


In [49]:
lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus[:-1],
                                           id2word = id2word,
                                           num_topics = 10,
                                           random_state = 100,
                                           update_every = 1,
                                           chunksize = 100,
                                           passes = 10,
                                           alpha = 'auto'
                                           )

In [50]:
test_doc = corpus[-1]

vector = lda_model[test_doc]
print(vector)

def Sort(sub_li):
    sub_li.sort(key = lambda x: x[1])
    sub_li.reverse()
    return(sub_li)

new_vector = Sort(vector)
print(new_vector)

[(0, 0.022351362), (1, 0.3340678), (2, 0.04829189), (3, 0.033661485), (5, 0.02003212), (6, 0.023813939), (7, 0.49504387), (9, 0.013045302)]
[(7, 0.49504387), (1, 0.3340678), (2, 0.04829189), (3, 0.033661485), (6, 0.023813939), (0, 0.022351362), (5, 0.02003212), (9, 0.013045302)]


In [ ]:
lda

In [53]:
lda_model.save("Models/test_models.model")

In [55]:
new_model = gensim.models.LdaModel.load("Models/test_models.model")


In [56]:
test_doc = corpus[-1]

vector = new_model[test_doc]
print(vector)

def Sort(sub_li):
    sub_li.sort(key = lambda x: x[1])
    sub_li.reverse()
    return(sub_li)

new_vector = Sort(vector)
print(new_vector)

[(0, 0.022351878), (1, 0.33406532), (2, 0.048295703), (3, 0.0336613), (5, 0.020032087), (6, 0.023814075), (7, 0.49504378), (9, 0.0130448975)]
[(7, 0.49504378), (1, 0.33406532), (2, 0.048295703), (3, 0.0336613), (6, 0.023814075), (0, 0.022351878), (5, 0.020032087), (9, 0.0130448975)]


## Visualizing Data

In [51]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
#vis

C:\Users\Whan\anaconda3\Lib\site-packages\sklearn\manifold\_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.002489  0.121471       1        1  22.036468
1      0.112607  0.316816       2        1  18.608935
7     -0.330086 -0.115819       3        1  15.260609
3     -0.192041  0.285610       4        1  10.698032
9     -0.120288 -0.265021       5        1   8.803994
0      0.054900 -0.104350       6        1   6.370586
8      0.107004 -0.315641       7        1   5.738666
5     -0.261655  0.088889       8        1   5.269013
6      0.300107 -0.147500       9        1   4.970423
4      0.331940  0.135547      10        1   2.243274, topic_info=          Term         Freq        Total Category  logprob  loglift
337     ghetto  2306.000000  2306.000000  Default  30.0000  30.0000
187     course  1371.000000  1371.000000  Default  29.0000  29.0000
663   remember  1085.000000  1085.000000  Default  28.0000  28.0000
852        use  1652.000000  1652.000000  Default  27.0000  27.0000
569       over   643.000000   643.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
912     action    25.367687   120.615741  Topic10  -5.1355   2.2381
957     arrive    28.791519   257.322010  Topic10  -5.0089   1.6070
350      great    26.182464   258.134967  Topic10  -5.1039   1.5088
547         of    25.126245   326.162344  Topic10  -5.1451   1.2338
1479    lawyer    22.902458   141.945552  Topic10  -5.2378   1.9730

[700 rows x 6 columns], token_table=       Topic      Freq            Term
term                                  
0          2  0.330322            able
0          3  0.004434            able
0          5  0.467772            able
0          6  0.039905            able
0          7  0.157402            able
...      ...       ...             ...
10173      2  0.970562  youth_movement
2551       2  0.016739         zionist
2551       3  0.050216         zionist
2551       6  0.100432         zionist
2551       9  0.820192         zionist

[1538 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 8, 4, 10, 1, 9, 6, 7, 5])